# Import Packages

In [ ]:
################################################################################
# NUMPY
# conda install numpy

import numpy as np

################################################################################
# MATPLOTLIB
# conda install matplotlib

# import matplotlib.pyplot as plt

################################################################################
# SEABORN
# conda install seaborn

# import seaborn as sns

################################################################################
# PYLTTB - Time Series Downsampling Using Largest-Triangle-Three-Buckets
# pip install pylttb

# from pylttb import lttb

################################################################################
# NEO
# pip install git+https://github.com/NeuralEnsemble/python-neo.git
# - AxoGraph support requires axographio to be installed: pip install axographio

# import neo

################################################################################
# QUANTITIES
# conda install quantities

import quantities as pq
pq.markup.config.use_unicode = True  # allow symbols like mu for micro in output
pq.mN = pq.UnitQuantity('millinewton', pq.N/1e3, symbol = 'mN');  # define millinewton

################################################################################
# ELEPHANT
# pip install elephant

# import elephant

################################################################################
# PANDAS
# conda install pandas

# import pandas as pd

################################################################################
# STATSMODELS
# conda install statsmodels

# import statsmodels.api as sm

################################################################################
# SPM1D - One-Dimensional Statistical Parametric Mapping
# pip install spm1d

# import spm1d

################################################################################
# EPHYVIEWER
# pip install git+https://github.com/jpgill86/ephyviewer.git@experimental
# - requires PyAV: conda install -c conda-forge av

import ephyviewer

################################################################################
# ParseMetadata
# - requires ipywidgets: conda install ipywidgets
# - requires yaml:       conda install pyyaml

from ParseMetadata import MetadataSelector

################################################################################
# ImportData

from ImportData import LoadAndPrepareData

################################################################################
# NeoUtilities

from NeoUtilities import NeoAnalogSignalRAUC#, NeoAnalogSignalDerivative, CausalAlphaKernel

################################################################################
# NeoToEphyviewerBridge

from NeoToEphyviewerBridge import NeoSegmentToEphyviewerSources#, PlotExampleWithEphyviewer

# IPython Magics

In [ ]:
# # make figures interactive and open in a separate window
# # %matplotlib qt

# # make figures interactive and inline
# %matplotlib notebook

# # make figures non-interactive and inline
# # %matplotlib inline

# Select Data Set

In [ ]:
# initial_selection = '2016-09-26_SBM-TAPE 001'
# initial_selection = '2018-02-06_IN-VIVO_JG-03 003'
# initial_selection = '2018-05-20_IN-VIVO_JG-07 002'
initial_selection = '2018-06-21_IN-VIVO_JG-08 002'
# initial_selection = '2018-06-24_IN-VIVO_JG-08 001'

metadata = MetadataSelector(initial_selection = initial_selection)
display(metadata)

# Import and Filter the Data

In [ ]:
blk, annotations_dataframe, epoch_encoder_dataframe, spikes_dataframe = LoadAndPrepareData(metadata)
# display(blk)
# display(annotations_dataframe)
# display(epoch_encoder_dataframe)
# display(spikes_dataframe)

In [ ]:
rauc_sigs = []
for sig in blk.segments[0].analogsignals:
    rauc = NeoAnalogSignalRAUC(sig, baseline = 0*sig.units, bin_duration = 0.1*pq.s)
    rauc.name = sig.name + ' RAUC'
    rauc_sigs.append(rauc)

# Ephyviewer

In [ ]:
# # construct a basic ephyviewer window using automagic functions

# seg = blk.segments[0]
# sources = ephyviewer.get_sources_from_neo_segment(seg)
# # sources = NeoSegmentToEphyviewerSources(seg)

# app = ephyviewer.mkQApp()
# win = ephyviewer.compose_mainviewer_from_sources(sources)
# win.show()
# app.exec_()
# # PlotExampleWithEphyviewer(sources)

In [ ]:
################################################################################
# DEFAULT PLOT SETTINGS

def defaultKeepSignal(sig):
    return (not sig.name.startswith('Analog Input #')) and (sig.name != 'Clock')

def defaultUnits(sig):
    mapping = {
        'V': 'uV', # convert voltages to microvolts
        'N': 'mN', # convert forces to millinewtons
    }
    mapping = {pq.Quantity(1, k).dimensionality.simplified: v for k, v in mapping.items()}
    return mapping.get(sig.units.dimensionality.simplified, sig.units)

def defaultYLim(sig):
    mapping = {
        'V': [-120, 120], # plot range for voltages
        'N': [ -10, 300], # plot range for forces
    }
    mapping = {pq.Quantity(1, k).dimensionality.simplified: v for k, v in mapping.items()}
    return mapping.get(sig.units.dimensionality.simplified, [-1, 1])

def setDefaultsForPlots(metadata, blk):
    sigs = blk.segments[0].analogsignals
    signalNameToBlockIndex = {sig.name:i for i, sig in enumerate(sigs)}

    if metadata['plots'] is None:
        metadata['plots'] = [{'channel': sig.name} for sig in sigs if defaultKeepSignal(sig)]

    plots = []
    for plot in metadata['plots']:
        index = signalNameToBlockIndex.get(plot['channel'], None)
        if index is None:
            print('Warning: removing plot with channel name "{}" because channel was not found in blk!'.format(plot['channel']))
        else:
            plot['index'] = index
            plot.setdefault('units',  defaultUnits(sigs[index]))
            plot.setdefault('ylim',   defaultYLim(sigs[index]))
            plot.setdefault('ylabel', sigs[index].name)
            plots.append(plot)
    metadata['plots'] = plots

    return metadata['plots']

In [ ]:
################################################################################
# DATA SOURCES

seg = blk.segments[0]
sigs = seg.analogsignals
sources = NeoSegmentToEphyviewerSources(seg)
# sources = ephyviewer.get_sources_from_neo_segment(seg)

# filter epoch encoder data out of generic epoch and event lists
# so they are not presented multiple times
sources['epoch'][0].all = [ep for ep in sources['epoch'][0].all if ep['name'] != 'Epoch Encoder']
sources['event'][0].all = [ev for ev in sources['event'][0].all if ev['name'] != 'Epoch Encoder']

################################################################################
# APP AND WINDOW

# create a new app
app = ephyviewer.mkQApp()

# create a window that will be populated with viewers
win = ephyviewer.MainViewer(
#     settings_name='test2', # remember settings (e.g. xsize) between sessions
    show_auto_scale = False,
    global_xsize_zoom = True,
    play_interval = 0.1, # refresh period in seconds
)
win.setWindowTitle(metadata['key'])

################################################################################
# PREPARE TRACE PARAMETERS

setDefaultsForPlots(metadata, blk)

signalNameToPlotIndex = {p['channel']:i for i, p in enumerate(metadata['plots'])}

################################################################################
# PREPARE SCATTER PLOT PARAMETERS

all_times = sigs[0].times.rescale('s').magnitude # assuming all AnalogSignals have the same sampling rate and start time
spike_indices = {}
spike_channels = {}
for st in seg.spiketrains:
    if 'channels' in st.annotations:
        c = []
        for channel in st.annotations['channels']:
            index = signalNameToPlotIndex.get(channel, None)
            if index is None:
                print('Note: Spike train {} will not be plotted on channel {} because that channel isn\'t being plotted'.format(st.name, channel))
            else:
                c.append(index)
        if c:
            spike_channels[st.name] = c
            spike_indices[st.name] = np.where(np.isin(all_times, st.times.magnitude))[0]

################################################################################
# TRACES WITH SCATTER PLOTS

sig_source = ephyviewer.AnalogSignalSourceWithScatter(
    signals = np.concatenate([sigs[p['index']].as_array(p['units']) for p in metadata['plots']], axis = 1),
    sample_rate = sigs[0].sampling_rate, # assuming all AnalogSignals have the same sampling rate
    t_start = sigs[0].t_start,           # assuming all AnalogSignals start at the same time
    channel_names = [p['ylabel'] for p in metadata['plots']],
    scatter_indexes = spike_indices,
    scatter_channels = spike_channels,
)
sources['signal'] = [sig_source]

trace_view = ephyviewer.TraceViewer(source = sources['signal'][0], name = 'signals')
trace_view.params['scatter_size'] = 5

win.add_view(trace_view)

trace_view.params['display_labels'] = True

# select a color scheme
trace_view.params_controller.combo_cmap.setCurrentText('Accent')
trace_view.params_controller.on_automatic_color()

# adjust plot range, scaling, and positioning
trace_view.params['ylim_max'] = 0.5
trace_view.params['ylim_min'] = -trace_view.source.nb_channel + 0.5
trace_view.params['scale_mode'] = 'by_channel'
for i, p in enumerate(metadata['plots']):
    ylim_span = np.ptp(p['ylim'])
    ylim_center = np.mean(p['ylim'])
    trace_view.by_channel_params['ch{}'.format(i), 'gain'] = 1/ylim_span # rescale [ymin,ymax] across a unit
    trace_view.by_channel_params['ch{}'.format(i), 'offset'] = -i - ylim_center/ylim_span # center [ymin,ymax] within the unit

################################################################################
# TRACES OF RAUC

sig_rauc_source = ephyviewer.InMemoryAnalogSignalSource(
    signals = np.concatenate([rauc_sigs[p['index']].as_array() for p in metadata['plots']], axis = 1),
    sample_rate = rauc_sigs[0].sampling_rate, # assuming all AnalogSignals have the same sampling rate
    t_start = rauc_sigs[0].t_start,           # assuming all AnalogSignals start at the same time
    channel_names = [p['ylabel'] + ' RAUC' for p in metadata['plots']],
)
sources['signal_rauc'] = [sig_rauc_source]

trace_rauc_view = ephyviewer.TraceViewer(source = sources['signal_rauc'][0], name = 'signals rauc')

win.add_view(trace_rauc_view)#, tabify_with = 'signals')

trace_rauc_view.params['display_labels'] = True
trace_rauc_view.params['display_offset'] = True

# select a color scheme
trace_rauc_view.params_controller.combo_cmap.setCurrentText('Accent')
trace_rauc_view.params_controller.on_automatic_color()

# adjust plot range
trace_rauc_view.params['ylim_max'] = 0.5
trace_rauc_view.params['ylim_min'] = -trace_rauc_view.source.nb_channel + 0.5
trace_rauc_view.params['scale_mode'] = 'by_channel'
for i, p in enumerate(metadata['plots']):
    ylim_span = np.median(rauc_sigs[p['index']].magnitude) * 10
    ylim_center = ylim_span / 2
    trace_rauc_view.by_channel_params['ch{}'.format(i), 'gain'] = 1/ylim_span # rescale [ymin,ymax] across a unit
    trace_rauc_view.by_channel_params['ch{}'.format(i), 'offset'] = -i - ylim_center/ylim_span # center [ymin,ymax] within the unit

################################################################################
# FREQUENCY

# freq_view = ephyviewer.TimeFreqViewer(source = trace_view.source, name = 'timefreqs')

# freq_view.params['scale_mode'] = 'by_channel'
# freq_view.params['nb_column'] = 1
# freq_view.params['colormap'] = 'gray'
# freq_view.params.param('timefreq')['deltafreq'] = 100
# freq_view.params.param('timefreq')['f_start'] = 1
# freq_view.params.param('timefreq')['f_stop'] = 1500

# freq_view.by_channel_params['ch0', 'visible'] = False
# freq_view.by_channel_params['ch1', 'visible'] = True
# freq_view.by_channel_params['ch2', 'visible'] = True
# freq_view.by_channel_params['ch3', 'visible'] = True
# freq_view.by_channel_params['ch4', 'visible'] = False

# # freq_view.params.param('timefreq')['normalisation'] = 1.5
# freq_view.by_channel_params['ch1', 'clim'] = 3
# freq_view.by_channel_params['ch2', 'clim'] = 5
# freq_view.by_channel_params['ch3', 'clim'] = 10

# win.add_view(freq_view)#, tabify_with = 'signals')

################################################################################
# SPIKE TRAINS

if sources['spike'][0].nb_channel > 0:

    spike_train_view = ephyviewer.SpikeTrainViewer(source = sources['spike'][0], name = 'spiketrains')
    win.add_view(spike_train_view)

    # select a color scheme
    spike_train_view.params_controller.combo_cmap.setCurrentText('Accent')
    spike_train_view.params_controller.on_automatic_color()

################################################################################
# EPOCHS

if sources['epoch'][0].nb_channel > 0:

    epoch_view = ephyviewer.EpochViewer(source = sources['epoch'][0], name = 'epochs')
    win.add_view(epoch_view)

    # select a color scheme
    epoch_view.params_controller.combo_cmap.setCurrentText('Accent')
    epoch_view.params_controller.on_automatic_color()

################################################################################
# EPOCH ENCODER

if metadata['epoch_encoder_file'] is not None:

    writable_epoch_source = ephyviewer.CsvEpochSource(
        filename = metadata['epoch_encoder_file'],
        possible_labels = metadata['epoch_encoder_possible_labels'],
    )

    epoch_encoder = ephyviewer.EpochEncoder(source = writable_epoch_source, name = 'epoch encoder')
    epoch_encoder.params['exclusive_mode'] = False
    win.add_view(epoch_encoder)

################################################################################
# VIDEO

if metadata['video_file'] is not None:

    video_source = ephyviewer.MultiVideoFileSource(video_filenames = [metadata['video_file']])
    if metadata['video_offset'] is not None:
        video_source.t_starts[0] += metadata['video_offset']
        video_source.t_stops[0]  += metadata['video_offset']
        video_source._t_start = max(min(video_source.t_starts), 0)
        video_source._t_stop  = max(video_source.t_stops)

    video_view = ephyviewer.VideoViewer(source = video_source, name = 'video')
    win.add_view(video_view, location = 'bottom', orientation = 'horizontal')

################################################################################
# EVENTS

if sources['event'][0].nb_channel > 0:

    event_list = ephyviewer.EventList(source = sources['event'][0], name = 'events')
    win.add_view(event_list, split_with = 'video')

################################################################################
# DATAFRAME

# if annotations_dataframe is not None:

#     data_frame_view = ephyviewer.DataFrameView(source = annotations_dataframe, name = 'table')
#     win.add_view(data_frame_view, tabify_with = 'events')

################################################################################
# LAUNCH

# show main window
win.set_xsize(40) # seconds
win.show()

# run Qapp
app.exec_()